In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from tensorflow import keras
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten, Conv1D, BatchNormalization, Activation, Dropout
from tensorflow.keras import Model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
import math



filename='wq_data1_norn_re.xlsx'
df=pd.read_excel(filename, engine='openpyxl')

#수질 입력 자료
o=df.to_numpy()
in_tot=o[:,3:15]

in_size=in_tot.shape
print("total size=", in_size)

#부영양화 지수 변수:COD
out_index=o[:,17]
out_indx_size=out_index.shape
print("total size=", out_indx_size)

codmx=13.7
chlmx=178.8
tpmx=0.332

xTrain, xTest, yTrain, yTest = train_test_split(in_tot, out_index, test_size=0.3)

tr_size=xTrain.shape
print("Train size=", tr_size)

vl_size=xTest.shape
print("Validation size=", vl_size)

tt_ss=len(in_tot)
tr_ss=len(xTrain)
fe_ss=len(xTrain[0])
vl_ss=len(xTest)


x1Train=np.reshape(xTrain, (tr_ss, fe_ss, -1))
x1Test=np.reshape(xTest, (vl_ss, fe_ss, -1))
print(x1Train.shape)
print(x1Test.shape)

y1Train=np.reshape(yTrain, (tr_ss))
y1Test=np.reshape(yTest,(vl_ss))
print(y1Train.shape)
print(y1Test.shape)

tot=np.reshape(in_tot, (tt_ss,fe_ss,-1))
print(tot.shape)

xxTrain=tf.convert_to_tensor(x1Train, dtype=tf.float32)
yyTrain=tf.convert_to_tensor(y1Train, dtype=tf.float32)
xxTest=tf.convert_to_tensor(x1Test, dtype=tf.float32)
yyTest=tf.convert_to_tensor(y1Test, dtype=tf.float32)
ttot=tf.convert_to_tensor(tot, dtype=tf.float32)

max_pool_1d = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='valid')


def spatial_attention(input_feature, name):
    kernel_size = 7
    kernel_initializer =tf.keras.initializers.variance_scaling()
    with tf.compat.v1.variable_scope(name):
        avg_pool = tf.reduce_mean(input_feature, axis=[2], keepdims=True)
        assert avg_pool.get_shape()[-1] == 1
        max_pool = tf.reduce_max(input_feature, axis=[2], keepdims=True)
        assert max_pool.get_shape()[-1] == 1
        concat = tf.concat([avg_pool, max_pool], 2)
        assert concat.get_shape()[-1] == 2

        concat = Conv1D(1, kernel_size, padding="same", strides=1)(concat)
        concat = tf.keras.layers.LeakyReLU()(concat)
        assert concat.get_shape()[-1] == 1
        concat = tf.sigmoid(concat, 'sigmoid')

    return concat


def regression_dilated_cnn(Model_input):
    # global xss1
    xs1=spatial_attention(Model_input, 's1')
    xss1=xs1*Model_input
    xx1= keras.layers.PReLU()(xss1)

    x = Conv1D(16, 3, padding='same', strides=1)(xx1)
    x = BatchNormalization()(x)
    x = keras.layers.PReLU()(x)

    x = Conv1D(32, 3, padding='same', strides=1)(x)
    x = BatchNormalization()(x)
    x = keras.layers.PReLU()(x)

    x = Conv1D(64, 3, padding='same', strides=1)(x)
    x = BatchNormalization()(x)
    x = keras.layers.PReLU()(x)

    x = Flatten()(x)
    x = Dense(2)(x)
    x = keras.layers.PReLU()(x)
    
    x = Dense(1)(x)

    output = Model(Model_input, x, name='regression_dilated_cnn')

    return output


model_inputs = keras.Input(shape=(fe_ss, 1))
AI_model = regression_dilated_cnn(model_inputs)
print(AI_model.summary())
opt=keras.optimizers.Adam(learning_rate=0.0001)
AI_model.compile(loss='mse', optimizer=opt)
AI_model.fit(xxTrain, yyTrain, epochs=3200, batch_size=16)
train_rs=AI_model.predict(xxTrain)
validation_rs=AI_model.predict(xxTest)
tot_rs=AI_model.predict(ttot)

xs11=spatial_attention(xxTrain,'s1')
xs11=xs11.numpy()
xs11=xs11.reshape((354, 12))
print(xs11.shape)
print(xs11.dtype)

## estimated variables
tp_tr_re=train_rs[:]*tpmx
tp_vl_re=validation_rs[:]*tpmx
tp_tot_re=tot_rs[:]*tpmx

## Observed variables
ob_tp_tr=yTrain[:]*tpmx
ob_tp_vl=yTest[:]*tpmx
ob_tp_tot=out_index[:]*tpmx

# Accuracy

rms_tp_tr=sqrt(mean_squared_error(ob_tp_tr,tp_tr_re))
rms_tp_vl=sqrt(mean_squared_error(ob_tp_vl,tp_vl_re))
rms_tp_tot=sqrt(mean_squared_error(ob_tp_tot,tp_tot_re))


def nse(predictions, targets):
    out=(1-(np.sum((predictions-targets)**2)/np.sum((targets-np.mean(targets))**2)))
    return out

nse_tp_tr=nse(tp_tr_re,ob_tp_tr)
nse_tp_vl=nse(tp_vl_re,ob_tp_vl)
nse_tp_tot=nse(tp_tot_re,ob_tp_tot)

r2_tp_tr=r2_score(ob_tp_tr, tp_tr_re)
r2_tp_vl=r2_score(ob_tp_vl, tp_vl_re)
r2_tp_tot=r2_score(ob_tp_tot, tp_tot_re)

tp_acc_tr=np.vstack((rms_tp_tr,nse_tp_tr,r2_tp_tr))
tp_acc_vl=np.vstack((rms_tp_vl,nse_tp_vl,r2_tp_vl))
tp_acc_tot=np.vstack((rms_tp_tot,nse_tp_tot,r2_tp_tot))


dd=pd.DataFrame(ob_tp_tr, columns=['TP_OBS_TR'])
dd['TP_TR']=tp_tr_re
dd.to_csv('model_tr_result.csv', index=False)


dr=pd.DataFrame(ob_tp_vl, columns=['TP_OBS_VL'])
dr['TP_VL']=tp_vl_re
dr.to_csv('model_vl_result.csv', index=False)

dw=pd.DataFrame(ob_tp_tot, columns=['TP_OBS_TOT'])
dw['TP_TOT']=tp_tot_re
dw.to_csv('model_tot_result.csv', index=False)

dq=pd.DataFrame(tp_acc_tr, columns=['TP_ACC_TR'])
dq['TP_ACC_VL']=tp_acc_vl
dq['TP_ACC_TOT']=tp_acc_tot
dq.to_csv('model_accuracy.csv', index=False)

dy=pd.DataFrame(xs11)
dy.to_csv('weight.csv',index=False)
